<a href="https://colab.research.google.com/github/slayerzeroa/Railway_Accident_BERT/blob/main/Tagtog_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install gspread
!pip install oauth2client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
# 라이브러리
import torch
from torch import nn, optim, tensor
from torch.utils.data import DataLoader, Dataset,TensorDataset, random_split
import torch.nn.functional as F

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, mean_squared_error

# from transformers import BertTokenizer

import json
import glob
import re
import os
import ast
import html
import requests

import pandas as pd

from google.colab import files

# Tagtog Log-In

In [34]:
#로그인 위치
url = "https://tagtog.com/-login"

#다운로드 위치
file_url = 'https://www.tagtog.com/slayerzeroa/Railway_BERT/-downloads/dataset-as-anndoc' 
o_file = 'Railway_Annotation.zip'  
if os.path.exists(o_file):
    os.remove(o_file)

#로그인 정보
login_info = {
    'loginid' : 'slayerzeroa@gmail.com',
    'password' : 'gyysxw5rU3NzrYX',
}

#로그인
with requests.Session() as s:
    login_req = s.post(url, data=login_info)
    r = s.get(file_url)

    with open(o_file,"wb") as output:
        output.write(r.content)

In [35]:
#압축파일 풀기
import zipfile
import shutil
folder_path = "./tagtog_relation_extraction/"

zip_ = zipfile.ZipFile("Railway_Annotation.zip")
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
zip_.extractall(folder_path)

In [36]:
import json
import glob
import re
import os
#폴더 경로
folder_name = "./tagtog_relation_extraction/Railway_BERT/"

#context list   # context == plain.html
context_name_list = os.listdir(folder_name + "plain.html/pool")
print(context_name_list)

#relation 폴더 경로   # relation == ann.json
relation_folder_paths = glob.glob(folder_name + "ann.json/master/pool")

#context 폴더 경로
# contexts_folders_paths = glob.glob(folder_name + "plain.html/pool/*")
contexts_folders_paths = [folder_name + "plain.html/pool/" + c for c in context_name_list]

#anntation_lenged 정보
annotations_legend = folder_name + "annotations-legend.json"
with open(annotations_legend,"r") as f:
    annotations_legend = json.load(f)


['a3DnzcVSvJKG25OeznRpiW5zLgi8-sample.pdf.plain.html', 'arqduk0rnomRe20HKEeXlVPK8qx8-20191022.pdf.plain.html', 'awUyBdL4lPpVlBOPjSzMhAuRk81a-20181208.pdf.plain.html', 'aVuARZFRfgJMD0E67Coa.7G8Z9Pe-20200226.pdf.plain.html', 'aPSRwvP5rtizz2O093I3oFX9uzjS-20180818.pdf.plain.html']


# Tagtog Annotation json File Preprocessing

In [37]:
def split_relation_start(text):
  text = text.split('|')[-1]
  start = text.split(',')[0]
  start = int(start)
  return start

In [67]:
'''
Loc : Sub, Obj 위치
Cls : Sub, Obj 클래스
Part : Sub, Obj html 상 문단
Sub : Sub 내용
Obj : Obj 내용
Label : 관계 있으면 1 없으면 0
Relation DataFrame Content
'''
for context_name, relation_folder, contexts_folder in zip(context_name_list, relation_folder_paths, contexts_folders_paths):
    # relation files와 context files 리스트 출력
    file_ids = [file_name for file_name in os.listdir(relation_folder)]
    file_nums = [ids.split("-")[1] for ids in file_ids]
    relation_files = [relation_folder + "/"+ file_id for file_id in file_ids]
    context_files = [contexts_folder for file_id in file_ids]
    Rel_df = pd.DataFrame()
    # Tagtog Annotation json 파일 데이터셋 전처리
    for relation_file, context_file, file_num in zip(relation_files, context_files, file_nums):
      with open(relation_file,"r") as f:
        relation = json.load(f)
        idx_list = []
        classid_list = []
        part_list = []
        text_list = []
        start_list = []

        for idx, content in enumerate(relation['entities']):
          idx_list.append(idx)
          classid_list.append(content['classId'])
          part_list.append(content['part'])
          text_list.append(content['offsets'][0]['text'])
          start_list.append(content['offsets'][0]['start'])
        
        df = pd.DataFrame(zip(idx_list, start_list, classid_list, part_list, text_list))
        df.columns = ['Index', 'Start','Class', 'Part', 'Text']

        doc_list = []
        loc_list = []
        cls_list = []
        part_list = []
        subject_list = []
        object_list = []
        label_list = []


        for rel in relation['relations']:
          subject = split_relation_start(rel['entities'][0])
          subject_class = df.loc[df['Start']==subject]['Class'].values[0]
          subject_part = df.loc[df['Start']==subject]['Part'].values[0]
          subject_text = df.loc[df['Start']==subject]['Text'].values[0]
          

          object = split_relation_start(rel['entities'][1])
          object_class = df.loc[df['Start']==object]['Class'].values[0]
          object_part = df.loc[df['Start']==object]['Part'].values[0]
          object_text = df.loc[df['Start']==object]['Text'].values[0]

          loc = str(subject) + '_' + str(object)
          cls = subject_class + '_' + object_class
          part = subject_part + '_' + object_part

          doc_list.append(str(file_num)[2:-13])
          loc_list.append(loc)
          cls_list.append(cls)
          part_list.append(part)
          subject_list.append(subject_text)
          object_list.append(object_text)
          label_list.append(1)
          Rel_df = Rel_df.append(pd.DataFrame(zip(doc_list, loc_list, cls_list, part_list, subject_list, object_list, label_list)))

Rel_df.columns = ['Doc', 'Loc', 'Cls', 'Part', 'Sub', 'Obj', 'Label']

print(Rel_df)

       Doc        Loc      Cls         Part  \
0   181208  4370_4249  e_6_e_4    s1p2_s1p2   
0   181208  4370_4249  e_6_e_4    s1p2_s1p2   
1   181208  4437_4249  e_6_e_4    s1p2_s1p2   
0   181208  4370_4249  e_6_e_4    s1p2_s1p2   
1   181208  4437_4249  e_6_e_4    s1p2_s1p2   
..     ...        ...      ...          ...   
2   200226     191_76  e_6_e_4  s1p17_s1p17   
0   200226    228_112  e_6_e_4    s1p5_s1p5   
1   200226    279_112  e_6_e_4    s1p5_s1p5   
2   200226     191_76  e_6_e_4  s1p17_s1p17   
3   200226     242_76  e_6_e_4  s1p17_s1p17   

                                                  Sub  \
0   청량신호소 및 강릉차량기지 연동검사 과정 에서 21A호, 21B호 선로전환기가 반대...   
0   청량신호소 및 강릉차량기지 연동검사 과정 에서 21A호, 21B호 선로전환기가 반대...   
1   청량신호소 21A호, 21B호 선로전환기를 설계변경하고 설치․시공ㆍ감리 과 정에 변...   
0   청량신호소 및 강릉차량기지 연동검사 과정 에서 21A호, 21B호 선로전환기가 반대...   
1   청량신호소 21A호, 21B호 선로전환기를 설계변경하고 설치․시공ㆍ감리 과 정에 변...   
..                                                ...   
2      기지관제원이 선로전환기 할출 및 할입 사 고에 대한 

In [68]:
print(Rel_df)

       Doc        Loc      Cls         Part  \
0   181208  4370_4249  e_6_e_4    s1p2_s1p2   
0   181208  4370_4249  e_6_e_4    s1p2_s1p2   
1   181208  4437_4249  e_6_e_4    s1p2_s1p2   
0   181208  4370_4249  e_6_e_4    s1p2_s1p2   
1   181208  4437_4249  e_6_e_4    s1p2_s1p2   
..     ...        ...      ...          ...   
2   200226     191_76  e_6_e_4  s1p17_s1p17   
0   200226    228_112  e_6_e_4    s1p5_s1p5   
1   200226    279_112  e_6_e_4    s1p5_s1p5   
2   200226     191_76  e_6_e_4  s1p17_s1p17   
3   200226     242_76  e_6_e_4  s1p17_s1p17   

                                                  Sub  \
0   청량신호소 및 강릉차량기지 연동검사 과정 에서 21A호, 21B호 선로전환기가 반대...   
0   청량신호소 및 강릉차량기지 연동검사 과정 에서 21A호, 21B호 선로전환기가 반대...   
1   청량신호소 21A호, 21B호 선로전환기를 설계변경하고 설치․시공ㆍ감리 과 정에 변...   
0   청량신호소 및 강릉차량기지 연동검사 과정 에서 21A호, 21B호 선로전환기가 반대...   
1   청량신호소 21A호, 21B호 선로전환기를 설계변경하고 설치․시공ㆍ감리 과 정에 변...   
..                                                ...   
2      기지관제원이 선로전환기 할출 및 할입 사 고에 대한 

#Google Spreadsheet 연동

In [69]:
# GOOGLE API KEY: json
key = files.upload()

Saving railway-tagtog-d24e6901ddb0.json to railway-tagtog-d24e6901ddb0.json


In [70]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

#json key file 위치
json_file_name = './railway-tagtog-d24e6901ddb0.json'

# json key file을 이용하여 접속
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

#구글 스프레드 시트 주소
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1cpt2ad2pw0N2tC1H3MPtRAaZzFmgCh2If0gHOQKEIIo/edit#gid=0"

# 스프레드시트 문서 가져오기
doc = gc.open_by_url(spreadsheet_url)
## gc.create(spreadsheet_name) # 스프레드시트 생성


#시트 선택하기
sheet_name = "annotation"
worksheet = doc.worksheet(sheet_name) #해당 시트가 있는 경우 불러오기
## 403 error가 뜰 경우, google sheets API를 활성시켜줘야 함

In [79]:
print(Rel_df.iloc[0, 4])

청량신호소 및 강릉차량기지 연동검사 과정 에서 21A호, 21B호 선로전환기가 반대로 표시되는 것을 확인하지 못한 것


In [80]:
# 시트의 모든 데이터 가져오기
values = worksheet.get_all_values()
header, rows = values[0], values[1:]
data = pd.DataFrame(rows, columns=header)
column_list = ['Doc', 'Loc', 'Cls', 'Part', 'Sub', 'Obj', 'Label']
data = data[column_list]
data.head()

# 워크시트 범위 설정
worksheet.resize(len(Rel_df)+1,10)
list_range = f"a2:g{len(Rel_df)+1}"
cell_list = worksheet.range(list_range)

# 내용 작성
for i in range(len(cell_list)//len(column_list)):
    cell_list[(7*i)].value = str(Rel_df.iloc[i, 0])
    cell_list[(7*i)+1].value = str(Rel_df.iloc[i, 1])
    cell_list[(7*i)+2].value = str(Rel_df.iloc[i, 2])
    cell_list[(7*i)+3].value = str(Rel_df.iloc[i, 3])
    cell_list[(7*i)+4].value = str(Rel_df.iloc[i, 4])
    cell_list[(7*i)+5].value = str(Rel_df.iloc[i, 5])
    cell_list[(7*i)+6].value = int(Rel_df.iloc[i, 6])
    
worksheet.update_cells(cell_list)

{'spreadsheetId': '1cpt2ad2pw0N2tC1H3MPtRAaZzFmgCh2If0gHOQKEIIo',
 'updatedRange': 'annotation!A2:G192',
 'updatedRows': 191,
 'updatedColumns': 7,
 'updatedCells': 1337}